In [1]:
from pyspark.sql import SparkSession
files_path = "/home/iceberg/playlist_data/*.json"
spark = (SparkSession
         .builder
         .appName("IcebergDemo")
         .getOrCreate()
        )

23/05/05 01:43:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from pyspark.sql.functions import explode, col, from_unixtime

df = (
    spark.read.json(files_path, multiLine=True)
    .select(explode('playlists').alias('playlist'))
    .select('playlist',
            explode('playlist.tracks').alias('track'))
    .select(col('playlist.name').alias('playlist_name'),
            col('playlist.pid').alias('playlist_id'),
            (col('playlist.pid') % 4).alias('partition_id'),
            col('playlist.collaborative').alias('playlist_collaborative'),
            col('playlist.num_tracks').alias('playlist_tracks'),
            col('playlist.num_albums').alias('playlist_albums'),
            col('playlist.num_followers').alias('playlist_followers'),
            from_unixtime(col('playlist.modified_at')).alias('playlist_modified_at'), 
            col('track.pos').alias('track_position'),
            col('track.artist_name').alias('track_artist_name'),
            col('track.track_uri').alias('track_uri'),
            col('track.artist_uri').alias('track_artist_uri'),
            col('track.track_name').alias('track_name'),
            col('track.album_uri').alias('track_album_uri'),
            (col('track.duration_ms')/1000).alias('track_duration'),
            col('track.album_name').alias('track_album_name'))
)

In [3]:
df.groupby('partition_id').count().show()

+------------+------+
|partition_id| count|
+------------+------+
|           0|416542|
|           1|415697|
|           3|415309|
|           2|422447|
+------------+------+



In [4]:
(df
 .sortWithinPartitions('partition_id')
 .writeTo('stage.spotify.playlists')
 .partitionedBy('partition_id')
 .create()
)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".    (0 + 4) / 9]
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
